In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import os
from tqdm import tqdm
import copy

In [6]:
os.chdir('../')

In [7]:
%pwd

'c:\\Users\\AJS\\Desktop\\DS Projects\\Scoliosis-Xray-Classification'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print(device) # cuda

cuda


In [8]:
# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Define data directory
data_dir = 'data'

# Load the dataset
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# Define the split sizes
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Apply the appropriate transforms to each subset
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']
test_dataset.dataset.transform = data_transforms['test']

# Create DataLoaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4),
    'test': DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=4)
}

# Get dataset sizes
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(test_dataset)
}

# Get class names
class_names = full_dataset.classes

print(f"Dataset sizes: {dataset_sizes}")
print(f"Class names: {class_names}")

Dataset sizes: {'train': 181, 'val': 38, 'test': 40}
Class names: ['Normal', 'Scol']


In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']} # NEEDS FIXING
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [9]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # Binary classification
model = model.to(device)

c:\Users\AJS\Desktop\DS Projects\Scoliosis-Xray-Classification\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\AJS\Desktop\DS Projects\Scoliosis-Xray-Classification\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\AJS/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


In [10]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
def train_model(model, criterion, optimizer, num_epochs=25):
    """
    Trains a given model using the specified criterion and optimizer for a specified number of epochs.
    Args:
        model (torch.nn.Module): The model to be trained.
        criterion (torch.nn.Module): The loss function used for training.
        optimizer (torch.optim.Optimizer): The optimizer used for updating the model's parameters.
        num_epochs (int, optional): The number of epochs to train the model. Defaults to 25.
    Returns:
        torch.nn.Module: The trained model.
    Raises:
        None
    """
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Use tqdm for progress tracking
            dataloader = tqdm(dataloaders[phase], desc=f'{phase.capitalize()} Epoch {epoch}/{num_epochs - 1}')

            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device).float().unsqueeze(1)

                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.sigmoid(outputs) > 0.5
                    loss = criterion(outputs, labels)

                    # Backpropagation + Optimization only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # Update tqdm progress bar
                dataloader.set_postfix({'Loss': loss.item()})

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    print(f'Best val Acc: {best_acc:4f}')
    model.load_state_dict(best_model_wts)
    return model

In [16]:
model = train_model(model, criterion, optimizer)

Epoch 0/24
----------


Train Epoch 0/24: 100%|██████████| 6/6 [00:11<00:00,  1.85s/it, Loss=1.14] 


train Loss: 0.5770 Acc: 0.7845


Val Epoch 0/24: 100%|██████████| 2/2 [00:04<00:00,  2.34s/it, Loss=26.6]


val Loss: 45.0270 Acc: 0.8158
Epoch 1/24
----------


...

val Loss: 0.0063 Acc: 1.0000
Epoch 24/24
----------


Train Epoch 24/24: 100%|██████████| 6/6 [00:05<00:00,  1.05it/s, Loss=0.000105]


train Loss: 0.0009 Acc: 1.0000


Val Epoch 24/24: 100%|██████████| 2/2 [00:04<00:00,  2.35s/it, Loss=7.83e-6]

val Loss: 0.0111 Acc: 1.0000
Best val Acc: 1.000000


In [17]:
model.eval()
running_corrects = 0

with torch.no_grad():
    # Use tqdm for progress tracking
    dataloader = tqdm(dataloaders['test'], desc='Testing')

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / len(test_dataset)
print(f'Test accuracy: {test_acc:.4f}')

Testing: 100%|██████████| 2/2 [00:04<00:00,  2.43s/it]

Test accuracy: 1.0000


In [26]:
from PIL import Image
import matplotlib.pyplot as plt

# Load the image
image_path = 'custom_tests/adam_scol_cropped.jpg'
image = Image.open(image_path)

# Apply the same transformations as used for validation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.Resize((224, 224)),  # Resize to the required input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Transform the image
input_image = transform(image)  # Remove unsqueeze(1) operation

# Move the image to the same device as the model
input_image = input_image.unsqueeze(0).to(device)  # Add batch dimension

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    # Get the model output
    output = model(input_image)
    # Apply sigmoid to get probabilities
    prob = torch.sigmoid(output).item()

# Interpret the output
prediction = 'Positive for scoliosis' if prob > 0.5 else 'Negative for scoliosis'

# Print the result
print(f'Prediction: {prediction} (Probability: {prob:.4f})')

# Display the image
# plt.imshow(image)
# plt.title(prediction)
# plt.axis('off')
# plt.show()

Prediction: Positive for scoliosis (Probability: 1.0000)
